In [18]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time


In [ ]:
class total_demo(nn.Module):

    def __init__(self) -> None:
        super(total_demo, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x


In [14]:
# 添加tensorboard                                     加载数据集
writer = SummaryWriter("./tensorboard/total_net")
train_data = torchvision.datasets.CIFAR10(root="./dataset", train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="./dataset", train=False, transform=torchvision.transforms.ToTensor(), download=True)

train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集长度为{}".format(train_data_size))          # ctrl+D：复制上一行
print("测试数据集长度为{}".format(test_data_size))

train_dataloader = DataLoader(train_data, 64)
test_dataloader = DataLoader(test_data,64)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Files already downloaded and verified
Files already downloaded and verified
训练数据集长度为50000
测试数据集长度为10000
cuda


In [22]:
# 实例化网络
lcj_CIFRA10 = total_demo()
# if torch.cuda.is_available():
#   lcj_CIFRA10 = lcj_CIFRA10.cuda()
lcj_CIFRA10 = lcj_CIFRA10.to(device)

# 定义损失函数
loss_func = nn.CrossEntropyLoss()
# if torch.cuda.is_available():
#   loss_func = loss_func.cuda()
loss_func = loss_func.to(device)

# 定义优化器
learning_rate = 0.01
# learning_rate = 1e-2
opti = torch.optim.SGD(lcj_CIFRA10.parameters(), lr=learning_rate)


total_train_step = 0
total_test_step = 0
epoch = 10

In [23]:
start_time = time.time()

# 预训练
for i in range(epoch):
    print("------------第{}轮训练-------------".format(i + 1))

    # 训练步骤开始
    lcj_CIFRA10.train()
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        # if torch.cuda.is_available():
        #   imgs = imgs.cuda()
        #   targets = targets.cuda()
        outputs= lcj_CIFRA10(imgs)
        loss = loss_func(outputs, targets)

        # 优化器
        opti.zero_grad()
        loss.backward()
        opti.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数：{}，Loss：{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
#         加了item不会输出tensor

    # 测试部分
    lcj_CIFRA10.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            # if torch.cuda.is_available():
            #   imgs = imgs.cuda()
            #   targets = targets.cuda()
            outputs = lcj_CIFRA10(imgs)
            loss = loss_func(outputs, targets)
            total_test_loss += loss

            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy


    print("整体测试集的loss：{}".format(total_test_loss))
    print("整体测试集的accuracy：{}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1

    torch.save(lcj_CIFRA10, "lcj_{}.pth".format(i))
    print("模型已保存")

end_time = time.time()
print("总共耗时：{}".format(end_time - start_time))

writer.close()

------------第1轮训练-------------
训练次数：100，Loss：2.1832711696624756
训练次数：200，Loss：2.0577707290649414
训练次数：300，Loss：1.8648574352264404
训练次数：400，Loss：1.732063889503479
训练次数：500，Loss：1.6909267902374268
训练次数：600，Loss：1.6384484767913818
训练次数：700，Loss：1.653573751449585
整体测试集的loss：329.7904052734375
整体测试集的accuracy：0.3042999804019928
模型已保存
------------第2轮训练-------------
训练次数：800，Loss：1.3517118692398071
训练次数：900，Loss：1.3225282430648804
训练次数：1000，Loss：1.3718667030334473
训练次数：1100，Loss：1.478894591331482
训练次数：1200，Loss：1.4298622608184814
训练次数：1300，Loss：1.2147213220596313
训练次数：1400，Loss：1.2308236360549927
训练次数：1500，Loss：1.4144335985183716
整体测试集的loss：248.3797149658203
整体测试集的accuracy：0.44039997458457947
模型已保存
------------第3轮训练-------------
训练次数：1600，Loss：1.0737258195877075
训练次数：1700，Loss：1.223677635192871
训练次数：1800，Loss：1.1744630336761475
训练次数：1900，Loss：1.215772271156311
训练次数：2000，Loss：1.4752529859542847
训练次数：2100，Loss：0.9777539968490601
训练次数：2200，Loss：0.9329079985618591
训练次数：2300，Loss：1.185781478881836
整